In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
# Read the pipe-delimited text file
df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')

# Save to CSV
df.to_csv('../data/data.csv', index=False)

print("Conversion completed: 'data.csv' created successfully.")

In [ ]:
df = pd.read_csv("../data/data.csv")
# Get number of columns
num_columns = df.shape[1]
print(f"Number of columns: {num_columns}")

# Get column data types
print("\nColumn data types:")
print(df.dtypes)

Data Summarization

In [ ]:
# Define numerical columns of interest
numeric_cols = [
    "TotalPremium",
    "TotalClaims",
    "CalculatedPremiumPerTerm",
    "SumInsured",
    "CustomValueEstimate",
    "NumberOfVehiclesInFleet",
    "CapitalOutstanding"
]

# Convert these columns to numeric, coercing errors to NaN
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Initial NaN count (before imputation)
nan_counts_before = df[numeric_cols].isna().sum()
print("Number of rows with NaN in each column (before imputation):")
print(nan_counts_before)

# Impute CapitalOutstanding with median
median_value = df["CapitalOutstanding"].median()
df["CapitalOutstanding"].fillna(median_value, inplace=True)

# Recalculate NaN count (after imputation)
nan_counts_after = df[numeric_cols].isna().sum()
print("\nNumber of rows with NaN in each column (after imputing CapitalOutstanding):")
print(nan_counts_after)

# Count rows with at least one NaN in numeric columns
num_nan_rows = df[df[numeric_cols].isna().any(axis=1)].shape[0]
print(f"\nNumber of rows with at least one NaN in numeric columns: {num_nan_rows}")

# Optional: Check how many columns in the entire DataFrame still have NaNs
cols_with_nan = df.isna().sum()
print("\nTotal columns still containing NaNs in the entire DataFrame:")
print(cols_with_nan[cols_with_nan > 0])

handle missing data

In [ ]:
#Drop Columns with Extremely High Missingness & Low Predictive Value
df.drop(columns=[
    "CustomValueEstimate", "NumberOfVehiclesInFleet",
    "WrittenOff", "Rebuilt", "Converted", "CrossBorder"
], inplace=True)


In [ ]:
#Categorical Variables — Impute with "Unknown" or Mode
categorical_cols = ["Bank", "AccountType", "MaritalStatus", "Gender", 
                    "VehicleType", "make", "Model", "bodytype", "NewVehicle"]

for col in categorical_cols:
    df[col].fillna("Unknown", inplace=True)

In [ ]:
#Numerical Variables — Impute with Median
num_cols = ["mmcode", "Cylinders", "cubiccapacity", "kilowatts", "NumberOfDoors"]

for col in num_cols:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)


In [ ]:
# Get number of columns
num_columns = df.shape[1]
print(f"Number of columns: {num_columns}")

In [ ]:
# Optional: Check how many columns in the entire DataFrame still have NaNs
cols_with_nan_after_handling = df.isna().sum()
print("\nTotal columns still containing NaNs in the entire DataFrame:")
print(cols_with_nan_after_handling[cols_with_nan_after_handling > 0])

In [ ]:
# Compute basic descriptive statistics
descriptive_stats = df[numeric_cols].describe().T  # Transpose for better readability

# Add variance and coefficient of variation (CV = std / mean)
descriptive_stats["variance"] = df[numeric_cols].var()
descriptive_stats["cv"] = descriptive_stats["std"] / descriptive_stats["mean"]

# Display results
print(descriptive_stats)

Data Structure

In [ ]:
# Display data types of each column
print("Column Data Types:")
print(df.dtypes)

# Summary of column types
print("\nSummary of Column Types:")
print(df.dtypes.value_counts())

# Identify columns by inferred type
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
bool_cols = df.select_dtypes(include=['bool']).columns.tolist()

# Display lists
print(f"\nCategorical Columns ({len(categorical_cols)}):\n{categorical_cols}")
print(f"\nNumerical Columns ({len(numerical_cols)}):\n{numerical_cols}")
print(f"\nBoolean Columns ({len(bool_cols)}):\n{bool_cols}")

# Check for possible datetime fields
print("\nChecking object columns for potential date fields:")
for col in categorical_cols:
    try:
        parsed = pd.to_datetime(df[col], errors='raise')
        print(f"✅ '{col}' can be converted to datetime.")
    except:
        continue

In [ ]:
df.isna().sum()

In [ ]:
# Define number of plots per row
plots_per_row = 3
num_cols = len(numerical_cols)
num_rows = math.ceil(num_cols / plots_per_row)

# Set figure size based on number of rows/columns
plt.figure(figsize=(plots_per_row * 6, num_rows * 4))
sns.set(style="whitegrid")

# Plot histograms in a grid layout
for idx, col in enumerate(numerical_cols, start=1):
    plt.subplot(num_rows, plots_per_row, idx)
    sns.histplot(df[col].dropna(), kde=True, bins=30, color='skyblue')
    plt.title(f"{col}")
    plt.xlabel("")
    plt.ylabel("")

plt.tight_layout()
plt.show()

In [ ]:
# Set style for plots
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# --- 2. Bar Charts for Categorical Variables ---
for col in categorical_cols:
    plt.figure()
    top_values = df[col].value_counts().nlargest(10)
    sns.barplot(x=top_values.index, y=top_values.values)
    plt.title(f"Top 10 Categories in '{col}'")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# --- 3. Bar Charts for Boolean Variables ---
for col in bool_cols:
    plt.figure()
    sns.countplot(x=df[col])
    plt.title(f"Distribution of Boolean Feature: '{col}'")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

Bivariate / Multivariate Analysis

In [ ]:
# 1. Convert 'TransactionMonth' to datetime
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')

# 2. Group by PostalCode and Month, then aggregate
grouped = df.groupby(['PostalCode', pd.Grouper(key='TransactionMonth', freq='M')])[
    ['TotalPremium', 'TotalClaims']
].sum().reset_index()

# 3. Scatter Plot: TotalPremium vs TotalClaims
plt.figure(figsize=(10, 6))
sns.scatterplot(data=grouped, x='TotalPremium', y='TotalClaims', hue='PostalCode', palette='tab20', legend=False)
plt.title("TotalPremium vs TotalClaims by PostalCode per Month")
plt.xlabel("Total Premium (monthly, per Zip)")
plt.ylabel("Total Claims (monthly, per Zip)")
plt.tight_layout()
plt.show()

# 4. Correlation Matrix: Numerical aggregates per PostalCode
postal_corr = grouped.groupby("PostalCode")[['TotalPremium', 'TotalClaims']].sum().corr()

# Plot heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(postal_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix: TotalPremium & TotalClaims (Aggregated by ZipCode)")
plt.tight_layout()
plt.show()

Data Comparison Over Geography

In [ ]:

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# --- 1. Compare CoverType distribution by Province ---
cover_geo = df.groupby(['Province', 'CoverType']).size().unstack().fillna(0)

cover_geo.plot(kind='bar', stacked=True)
plt.title("CoverType Distribution by Province")
plt.ylabel("Number of Policies")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- 2. Compare average TotalPremium by Province ---
premium_geo = df.groupby('Province')['TotalPremium'].mean().sort_values()

premium_geo.plot(kind='bar', color='teal')
plt.title("Average Total Premium by Province")
plt.ylabel("Average Premium")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- 3. Compare most popular Auto Make by Province (Top 5 Makes only) ---
top_makes = df['make'].value_counts().nlargest(5).index.tolist()
df_top_makes = df[df['make'].isin(top_makes)]

make_geo = df_top_makes.groupby(['Province', 'make']).size().unstack().fillna(0)

make_geo.plot(kind='bar', stacked=True)
plt.title("Top 5 Auto Makes Distribution by Province")
plt.ylabel("Number of Vehicles")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()